# **To define a trader**

#### stockait는 핵심 개념인 `Trader`라는 객체가 있다.
#### 하나의 트레이더 안에는 데이터셋과 인공지능 모델을 포함하고, 모델 학습을 수행한다.
#### 또한 주식 거래 (매수/매도) 시뮬레이션도 트레이더를 통해 가능하다.  

### 이 파일에서는 이러한 `Trader`에 대한 설명과 예시를 보여준다.

<br>

## **Trader의 기능**
#### Trader는 크게 3가지 기능을 갖고 있다.

![img](../../image/trader_1.png)


#### 트레이더 객체는 1) 매수/매도 정보, 인공지능 모델 정보를 저장하며, 2) 데이터셋을 저장하고, 추가적으로 3) 이름, 레이블 정보를 저장한다.
#### 아래에서 이러한 3가지 기능을 하나씩 살펴보고, 객체를 직접 정의하는 예시에서 더 자세히 보도록 한다.

<br>

## **첫째, Trader object Definitions** 
#### Trader / Buyer / Seller 객체를 정의하는 것은 Trader의 핵심 기능이다.

![img](../../image/trader_2.png)

### **1)`trader.buyer`** 
buyer의 두가지 형태의 객체를 리스트로 저장한다. ([conditional_buyer, machine learning_buyer]).

#### **1.1)`conditional_buyer`** 
데이터셋의 필터링 조건으로 매수를 결정하는 객체. 

#### **1.2)`machinelearning_buyer`** 
머신러닝 모델의 예측확률을 통해 매수를 결정하는 객체.

<br>

### **2)`trader.seller`** 
seller의 객체를 저장한다.

#### **2.1)`SubSeller`**
매수한 모든 종목을 다음날에 모두 매도하는 객체.

<br> 

## **둘째, Save datasets**

#### 트레이더 객체 안에는 모델 학습/검증/시험 및 주식 매매를 위한 데이터셋이 저장된다.

![img](../../image/trader_3.png)

<br>

## **셋째, Save Trader's Information**
#### 트레이더를 구분하기 위한 `name` 속성과, 레이블 정보를 넣어주는 `label`속성을 저장하는 기능이다.

![img](../../image/trader_4.png)

-------- 


<br>

#### 아래에서는 Trader를 정의하는 과정을 자세히 들여다 보기 위해, LightGBM 모델을 사용하여 예시로 Trader를 정의한다.

<br>

#### 먼저, 트레이더들을 담을 빈 리스트를 하나 생성한다.

In [ ]:
lst_trader = [] 


<br>

### **ConditionalBuyer 객체와 sampling 함수 정의** 
#### ConditionalBuyer에서 주가 데이터를 필터링 할 조건을 설정하는 함수 sampling을 직접 정의한다. 종가 변화율이 30% 이상이면 제외하며, 종가 변화율이 너무 낮은 -0.05 ~ 0.05 인 데이터는 제외하고, 거래대금이 10억 이상인 데이터셋만을 사용하도록 조건을 생성했다. 

In [ ]:
# conditional_buyer: Object that determines acquisition based on data filtering conditions 
b1_lg = sai.ConditionalBuyer()

def sampling1(df): # Create a conditional function
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) # Remove exceptions that exceed upper and lower limits
    condition2 = df.D0_trading_value >= 1000000000 # condition 1: Transaction amount of more than 1 billion won 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) # condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_lg.condition = sampling1  # Define the condition function directly (sampling1) and store it in the condition property 

### **MachinelearnigBuyer 객체 정의** 
#### Machinelearning Buyer는 인공지능 모델 학습을 담당한다. 인공지능 모델과 관련된 정보들은 모두 Machinelearning Buyer 객체의 속성에 넣어줘야 한다. 
#### 모델을 정의하는 것은 여느 패키지의 사용법과 같고, 정의한 모델 객체를 machinelearning Buyer의 `algorithm` 속성에 넣어주면 된다.

In [ ]:
from lightgbm import LGBMClassifier

# machinelearning_buyer: Object that determines acquisition by machine learning model
b2_lg = sai.MachinelearningBuyer()

# Save user-defined models to algorithm properties
scale_pos_weight = round(72/28 , 2)
params = {  'random_state' : 42,
            'scale_pos_weight' : scale_pos_weight,
            'learning_rate' : 0.1, 
            'num_iterations' : 1000,
            'max_depth' : 4,
            'n_jobs' : 30,
            'boost_from_average' : False,
            'objective' : 'binary' }

b2_lg.algorithm =  LGBMClassifier( **params )

#### 이로써 Buyer 의 서브 객체들인 Conditional Buyer와 Machinelearning Buyer를 정의하였다.
### **SubSeller 객체 정의** 
#### SubSeller 객체는 매수한 주식을 매수 날짜 다음날에 모두 되파는 방법만을 제공하고있다. 이러한 SubSeller 객체를 하나 생성해준다. 

In [ ]:
# SubSeller: Object that determines selling all of the following days
sell_all = sai.SubSeller() 

### **트레이더 / Buyer / Seller 객체 정의** 
#### 트레이더 객체에는 트레이더를 구분하기 위한 `name` 속성, 종속변수를 설정하는 `label` 속성, Buyer, Seller 객체를 저장하는 `buyer` 속성과 `seller` 속성이 있다. 

In [ ]:
# Trader Object   
t1 = sai.Trader()
t1.name = 'saiLightGBM' # Trader's name
t1.label = 'class&0.02' # Set the Trader dependent variable (do not set if it is regression analysis) 
t1.buyer = sai.Buyer([b1_lg, b2_lg]) # [ conditional buyer, machinelearning buyer ] 
t1.seller = sai.Seller(sell_all)

lst_trader.append(t1)

#### 여기서 label을 보면, "class&0.02"라고 작성되어있는데, **classification** 문제이며, **다음날 종가 변화율**이 **0.02** 이상이면 **1**, 그렇지 않으면 **0**으로 이진분류한다는 뜻이다. 분류로 하고싶지 않으면 "reg"라고 설정하여, 다음날 종가 변화율 (next_change) 자체를 종속변수로 사용할 수 있다. 
#### buyer에는 Buyer 객체를 호출할 때 리스트 안에 위에서 정의했던 conditional buyer와 machinelearning buyer를 넣고, seller는 바로 SubSeller 객체를 넣어준다. 
#### 마지막으로, 처음에 만든 lst_trader에 트레이더를 추가한다. 

<br> 

#### 이로써 Trader 객체를 모두 정의하였다. **(딥러닝 모델을 정의하는 것은 추가적으로 요구되는 속성들이 있으므로, 아래의 모델 예시 (ex7, ex8)를 참고하거나, `02-1.tensorflow_example.ipynb`, `02-2.pytorch_example.ipynb` 파일에서 자세한 설명을 확인할 수 있다.**

<br>

#### 아래에는 여러 머신러닝 모델을 정의한 예시를 보여준다.

<br>

### **ex1) LightGBM** 

In [ ]:
from lightgbm import LGBMClassifier

# conditional_buyer: Object that determines acquisition based on data filtering conditions 
b1_lg = sai.ConditionalBuyer()

def sampling1(df): # Create a conditional function
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) # Remove exceptions that exceed upper and lower limits
    condition2 = df.D0_trading_value >= 1000000000 # condition 1: Transaction amount of more than 1 billion won 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) # condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_lg.condition = sampling1  # Define the condition function directly (sampling1) and store it in the condition property 


# machinelearning_buyer: Object that determines acquisition by machine learning model
b2_lg = sai.MachinelearningBuyer()

# Save user-defined models to algorithm properties
scale_pos_weight = round(72/28 , 2)
params = {  'random_state' : 42,
            'scale_pos_weight' : scale_pos_weight,
            'learning_rate' : 0.1, 
            'num_iterations' : 1000,
            'max_depth' : 4,
            'n_jobs' : 30,
            'boost_from_average' : False,
            'objective' : 'binary' }

b2_lg.algorithm =  LGBMClassifier( **params )


# SubSeller: Object that determines selling all of the following days
sell_all = sai.SubSeller() 


# Trader Object   
t1 = sai.Trader()
t1.name = 'saiLightGBM' # Trader's name
t1.label = 'class&0.02' # Set the Trader dependent variable (do not set if it is regression analysis) 
t1.buyer = sai.Buyer([b1_lg, b2_lg]) # [ conditional buyer, machinelearning buyer ] 
t1.seller = sai.Seller(sell_all)

lst_trader.append(t1)

<br>

### **ex2) XGBoost** 

In [ ]:
from xgboost import XGBClassifier

b1_xgb = sai.ConditionalBuyer() 

def sampling2(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_xgb.condition = sampling2


b2_xgb = sai.MachinelearningBuyer()  

scale_pos_weight = round(72/28 , 2)
b2_xgb.algorithm = XGBClassifier(random_state = 42,
                   n_jobs=30,
                   scale_pos_weight=scale_pos_weight,
                   learning_rate=0.1,
                   max_depth=4,
                   n_estimators=1000,
                   )  

sell_all = sai.SubSeller()


t2 = sai.Trader()
t2.name = 'saiXGboost' 
t2.label = 'class&0.02' 
t2.buyer = sai.Buyer([b1_xgb, b2_xgb])
t2.seller = sai.Seller(sell_all) 

lst_trader.append(t2) 

<br>

### **ex3) LogisticRegression** 

In [ ]:
from sklearn.linear_model import LogisticRegression

b1_lr = sai.ConditionalBuyer()

def sampling3(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_lr.condition = sampling3


b2_lr = sai.MachinelearningBuyer()  

b2_lr.algorithm = LogisticRegression()


sell_all = sai.SubSeller() 


t3 = sai.Trader()
t3.name = 'saiLogisticRegression'  
t3.label = 'class&0.02' 
t3.buyer = sai.Buyer([b1_lr, b2_lr]) 
t3.seller = sai.Seller(sell_all)

lst_trader.append(t3) 

<br> 

### **ex4) Support Vector Machine**

In [ ]:
from sklearn.svm import SVC

b1_sv = sai.ConditionalBuyer()

def sampling4(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_sv.condition = sampling4 


b2_sv = sai.MachinelearningBuyer()  

b2_sv.algorithm = SVC() 


sell_all = sai.SubSeller() 


t4 = sai.Trader()
t4.name = 'saiSupportVectorMachine'  
t4.label = 'class&0.02' 
t4.buyer = sai.Buyer([b1_sv, b2_sv]) 
t4.seller = sai.Seller(sell_all)

lst_trader.append(t4) 

<br>

### **ex5) Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

b1_dt = sai.ConditionalBuyer()

def sampling5(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_dt.condition = sampling5 


b2_dt = sai.MachinelearningBuyer()  

b2_dt.algorithm = DecisionTreeClassifier() 


sell_all = sai.SubSeller() 


t5 = sai.Trader()
t5.name = 'saiDecisionTree'  
t5.label = 'class&0.02' 
t5.buyer = sai.Buyer([b1_dt, b2_dt]) 
t5.seller = sai.Seller(sell_all)

lst_trader.append(t5) 

<br>

### **ex6) RandomForest** 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

b1_rf = sai.ConditionalBuyer()

def sampling6(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_rf.condition = sampling6 


b2_rf = sai.MachinelearningBuyer()  

b2_rf.algorithm = RandomForestClassifier() 


sell_all = sai.SubSeller() 


t6 = sai.Trader()
t6.name = 'saiDecisionTree'  
t6.label = 'class&0.02' 
t6.buyer = sai.Buyer([b1_rf, b2_rf]) 
t6.seller = sai.Seller(sell_all)

lst_trader.append(t6) 

----------


<br>

### **딥러닝 모델**

#### 딥러닝 모델을 사용하기 위해서는 2차원이었던 데이터 구조를 3차원으로 변환해주어야 한다. 따라서 `data_transform` 속성이 추가된다. 
#### 어떤 프레임워크를 사용하느냐에 따라 `framework` 속성을 넣어줘야 한다.
#### tensorflow의 경우에는 하이퍼 파라미터나 콜백함수를 `params`에 넣어주어야 하고, 
#### pytorch의 경우에는 `params` 속성과 더불어 gpu, cpu를 설정하는 `device`속성이나, 손실함수와 옵티마이저를 설정해주는 `optim` 속성도 추가적으로 요구된다.

<br>

#### **(이러한 딥러닝 모델에 대해서는 `02-1.tensorflow_example.ipynb`, `02-2.pytorch_example.ipynb` 파일에서 더 자세한 설명을 확인할 수 있다.)**

<br>

### **ex7) LSTM - keras**  

In [ ]:
b1_ls = sai.ConditionalBuyer()

def sampling1(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = (df.D0_Close * df.D0_Volume) >= 100000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3 
    return condition

b1_ls.condition = sampling1

b2_ls = sai.MachinelearningBuyer()

# User-defined functions (users who want deep learning modeling)
def transform1(data): # A function that converts into a two-dimensional structure / data: list (lst_time_series)
    data_2d = []
    n_col = int(len(data[0]) / 10) 
    for row in data:      
        data_2d.append([])
        for i in range(0, len(row), n_col):
            data_2d[-1].append(row[i:i+n_col])
    
    return np.array(data_2d)
    

# Directly define a two-dimensional structure transformation function (transform) and store it in the data_transform property
b2_ls.data_transform = transform1 

from keras.callbacks import EarlyStopping

# deep learning framework 
b2_ls.framework = "tensorflow"

# parameters for model.fit() 
early_stopping = EarlyStopping(monitor='val_loss', patience=7)
b2_ls.params = {
    "epochs": 20, 
    "batch_size": 64,
    "callbacks": [early_stopping]
    }

#model.add(keras.layers.Dropout(0.2))
# defines a model 
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(10, 48)))
model.add(keras.layers.LSTM(128, activation='selu', return_sequences=True))
model.add(keras.layers.LSTM(64, activation='selu', return_sequences=True))
model.add(keras.layers.LSTM(32, activation='selu', return_sequences=False))
model.add(keras.layers.Dense(1, activation='sigmoid'))
    
model.compile(optimizer=keras.optimizers.Adam(
    # learning_rate=keras.optimizers.schedules.ExponentialDecay(0.05,decay_steps=100000,decay_rate=0.96)), 
    learning_rate = 0.001), 
    loss="binary_crossentropy",
    metrics=['accuracy'])

b2_ls.algorithm = model

sell_all = sai.SubSeller() 

t7 = sai.Trader()
t7.name = 'saiLSTM_tf' 
t7.label = 'class&0.02' 
t7.buyer = sai.Buyer([b1_ls, b2_ls]) 
t7.seller = sai.Seller(sell_all)

lst_trader.append(t4)

<br>

### **ex8) LSTM - PyTorch**  

In [ ]:
import torch
import torch.nn as nn
import numpy as np

b1_lspt = sai.ConditionalBuyer()

def sampling5(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3)
    condition2 = (df.D0_Close * df.D0_Volume) >= 1000000000 
    # condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition4 = (df.D0_CCI <= -100)
    condition = condition1 & condition2 & condition4  
    return condition

b1_lspt.condition = sampling5

b2_lspt = sai.MachinelearningBuyer()

# ⭐ User-defined functions (users who want deep learning modeling)
def transform(data): # A function that converts into a two-dimensional structure / data: list (lst_time_series)
    data_2d = []
    n_col = int(len(data[0]) / 10) 
    for row in data:      
        data_2d.append([])
        for i in range(0, len(row), n_col):
            data_2d[-1].append(row[i:i+n_col])
    
    return np.array(data_2d)
    

# Directly define a two-dimensional structure transformation function (transform) and store it in the data_transform property
b2_lspt.data_transform = transform 

# framework 
b2_lspt.framework = "pytorch"

# devcice  
b2_lspt.device = device 

##### LSTM Model Definition #####
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # hidden state와 cell state의 값을 0으로 초기화 함 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # hidden state 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # cell state 
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        # out = torch.sigmoid(out) 
                      
        return out
    
                      
# learning rate 
learning_rate = 0.01

# create a model 
input_size, hidden_size, num_layers, output_size = 48, 64, 12, 1 
model = LSTM(input_size, hidden_size, num_layers, output_size)   

b2_lspt.algorithm = model

# hyper parameters for model fitting 
b2_lspt.params = {
    "epochs": 20, 
    "batch_size": 64,
    }


pos_weight = torch.tensor([7.0], device=device)

# loss function & optimizer 
b2_lspt.optim = {
    "criterion": nn.BCEWithLogitsLoss(pos_weight=pos_weight), 
    "optimizer": torch.optim.Adam(model.parameters(), lr=learning_rate)  
    }


sell_all = sai.SubSeller() 


t8 = sai.Trader()
t8.name = 'saiLSTM_pt' 
t8.label = 'class&0.02' 
t8.buyer = sai.Buyer([b1_lspt, b2_lspt]) 
t8.seller = sai.Seller(sell_all)

lst_trader.append(t8)

<br>

#### 이렇게 8가지의 트레이더들을 lst_trader에 넣으면, 각각의 모델을 한 번에 학습할 수 있고, 주식 매매 시뮬레이션을 진행할 수 있다.  
#### 여러 모델을 사용하여 모델 학습 및 주식 매매 시뮬레이션을 수행하는 예시는 `tutorials/03.different_models.ipynb` 파일에서 확인할 수 있다.